<a href="https://colab.research.google.com/github/heinsense2/AIO_CaseStudy/blob/main/notebooks/fathomnet_background_imgs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download background images from the FathomNet.

This notebook finds and downloads images from FathomNet to be used as background images during training.

Requirements: python 3.7 or greater and fathomnet.py

This notebook is in no way exhaustive and is not to be used as is. It shows how background images are found and downloaded. Changes are required depending on your particular data setup.



In [ ]:
# Assuming the data is available on Google drive
from google.colab import drive
drive.mount('/content/drive/MyDrive')

In [ ]:
# Change the location of this script depending on how your paths are set up
from fathomnet.api import xapikey,boundingboxes,images,taxa,geoimages
from fathomnet.models import AImageDTO, ABoundingBoxDTO, GeoImageConstraints, GeoImageConstraintsCount

import os
import ssl
import requests
import time
import random
from urllib.request import urlretrieve

In [ ]:
# Authentication key is different per user, can find specific ones 
# on the fathomnet website under account > API key
xapikey.auth('piACmngefKuKNJUDBk1VXRaHqKdaXD')

ssl._create_default_https_context = ssl._create_unverified_context

concepts = ["Chiroteuthis calyx", "Dosidicus gigas", "Gonatus onyx", "Sebastes", "Sebastes diploproa", "Sebastes melanostomus", "Sebastolobus"]

# BackgorundBackground images for domains pre_2012 and post_2012. 
data_dir1 = "pre_2012"
data_dir2 = "post_2012" 
os.makedirs(data_dir1, exist_ok = True)
os.makedirs(data_dir2, exist_ok = True)

# Using GeoImageConstraint dataclass create image domains based on date split
constraint_pre2012 = GeoImageConstraints(taxaProviderName='mbari', endTimestamp="2011-12-31T23:59:59.999Z")
constraint_post2012 = GeoImageConstraints(taxaProviderName='mbari', startTimestamp="2012-01-01T12:00:00.000Z")
IMAGESPRE = images.find(constraint_pre2012)
IMAGESPOST = images.find(constraint_post2012)

#create random sample of 250 (approx. 10% of individual domain sizes)
sample_size = 250 #value can be adjusted to match the size of your training/testing dataset
RANDOMPRE = random.sample(IMAGESPRE, sample_size)
RANDOMPOST = random.sample(IMAGESPOST, sample_size)

In [ ]:
def checkAndClearConcepts(SAMPLEIMGS, ALLIMGS):
    for img in SAMPLEIMGS:
        if isinstance(img,AImageDTO):
            bb = img.boundingBoxes
            if bb is not None:
                for item in bb:
                    if concepts.count(item.concept) > 0:
                        SAMPLEIMGS.remove(img)
                        SAMPLEIMGS.append(random.sample(ALLIMGS, 1))
                        break
        if isinstance(img, list):
            for img_record in img: 
                bb = img_record.boundingBoxes
                if bb is not None:
                    for item in bb:
                        if concepts.count(item.concept) > 0:
                            SAMPLEIMGS.remove(img)
                            SAMPLEIMGS.append(random.sample(ALLIMGS, 1))
                            break
                    else:
                        continue
                    break

def downloadImg(img_record, direc):
    url = img_record.url 
    extension = os.path.splitext(url)[-1]
    img_filename = os.path.join(direc, img_record.uuid + extension)
    urlretrieve(url, img_filename)
    return img_record.uuid + extension

# Check if sample contains any images of the classes in question, if so remove and replace
checkAndClearConcepts(RANDOMPRE, IMAGESPRE)
checkAndClearConcepts(RANDOMPOST, IMAGESPOST)

# Download images 
for img in RANDOMPRE:
    if isinstance(img,AImageDTO):
        downloadImg(img, data_dir1)
    if isinstance(img,list):
        for img_record in img: 
            downloadImg(img_record, data_dir1)
    
for img in RANDOMPOST:
    if isinstance(img,AImageDTO):
        downloadImg(img, data_dir2)
    if isinstance(img,list):
        for img_record in img: 
            downloadImg(img_record, data_dir2)